In [53]:
import sys
import numpy as np
from qiskit import *

sys.path.append('..')
from pyquopt import *

### Define Circuit of Interest

In [54]:
ghz_x = QuantumCircuit(2)
ghz_x.h(0)
ghz_x.cx(0, 1)
ghz_x.x(0)
ghz_x.draw()

┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘

In [55]:
ghz_x_transpiled = transpile(ghz_x, basis_gates=['sx', 'rz', 'cx'], optimization_level=3)

In [56]:
ghz_x_transpiled.draw()

global phase: π/4
     ┌─────────┐┌────┐┌─────────┐     ┌────┐┌────┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──┤ √X ├┤ √X ├
     └─────────┘└────┘└─────────┘┌─┴─┐└────┘└────┘
q_1: ────────────────────────────┤ X ├────────────
                                 └───┘

### Set Optimization Hyperparameters

In [57]:
num_qubits = 2
num_mq_instructions = 1  # want to explore circuits with one multi-qubit gate
num_params = 3 * num_qubits * (num_mq_instructions + 1)  # formula explained in MICRO paper

mq_dict = {
    0: TwoGates.CR01,
}

In [58]:
alpha = 0.30  # penalty for non-standard angles
gamma = 1e-5   # penalty for large angles
non_fixed_params = np.array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]) # leave all parameters unfixed
fixed_params_vals = np.array([0, 0, 0, 0, 0, 0, np.pi/2, 0, np.pi, np.pi, 0, np.pi])  # unfixed parameters should have a "fixed" value of 0

In [59]:
mq_instructions = [0]

### Solve Optimization Problem

In [60]:
optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=TwoGates.GHZ,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)

In [61]:
opt_params, opt_val = optimizer.find_parameters_least_squares(20)

In [62]:
opt_params_rounded = round_params(opt_params)
print(opt_params_rounded)

[180. 225. 135.  90. 270.  90.  90.   0. 180. 180.   0. 180.]


### Verify Implementation Accuracy

In [63]:
ub = UnitaryBuilder(num_qubits, mq_instructions, mq_dict)

In [64]:
implementation_matrix = ub.build_unitary(opt_params_rounded * np.pi / 180)

In [65]:
print(get_unitary_infidelity(TwoGates.GHZ, implementation_matrix, 4))

2.220446049250313e-16


In [66]:
implementation_matrix

array([[ 7.07106781e-01-9.81307787e-17j, -7.39139728e-17+1.28746949e-16j,
         7.07106781e-01-1.84726384e-16j, -1.26816328e-17+6.75146087e-17j],
       [ 1.26816328e-17+1.09120793e-16j,  7.07106781e-01-3.22109474e-16j,
         7.39139728e-17+4.78884530e-17j,  7.07106781e-01-4.08705080e-16j],
       [ 7.39139728e-17+8.94946372e-17j,  7.07106781e-01-2.23978696e-16j,
        -1.26816328e-17-2.82622972e-17j, -7.07106781e-01+3.10574301e-16j],
       [ 7.07106781e-01-2.74766180e-16j, -1.26816328e-17+6.98684814e-17j,
        -7.07106781e-01+3.61361786e-16j,  7.39139728e-17-8.63614149e-18j]])